In [2]:
import pandas as pd

df = pd.read_pickle('../data/clean_df.pkl')
print(len(df), " individus")


print(df['Departement'].nunique()," départements différents")
df.head(15)



In [37]:
import folium
from folium.plugins import HeatMap
from branca.colormap import linear

# Créez une liste de listes contenant les coordonnées [latitude, longitude] et la valeur "Valeur fonciere"
heat_data = [[row['latitude'], row['longitude'], row['Valeur_fonciere']]
             for index, row in df.iterrows()]

# Créez la carte
my_map = folium.Map(location=[46.603354, 1.888334], zoom_start=6)

# Créez une colormap linéaire basée sur la colonne "Valeur fonciere"
colormap = linear.YlOrRd_09.scale(
    min(df['Valeur_fonciere']), max(df['Valeur_fonciere']))

# Ajoutez la carte de chaleur à partir des données
HeatMap(heat_data,
        min_opacity=0.2,
        max_val=max(df['Valeur_fonciere']),
        radius=15,
        blur=10,
        max_zoom=1,
        gradient={0.4: 'blue', 0.65: 'lime', 1: 'red'}).add_to(my_map)

# Ajoutez une légende basée sur la colormap
colormap.caption = "Valeur fonciere"
colormap.add_to(my_map)

# Enregistrez la carte en tant que fichier HTML
my_map.save('map.html')


C:\Users\nagrimault\AppData\Local\Temp\ipykernel_11880\449697079.py:17: UserWarning: The `max_val` parameter is no longer necessary. The largest intensity is calculated automatically.
  HeatMap(heat_data,


In [41]:
import folium
from folium.plugins import HeatMap
from folium import Choropleth, GeoJson
import pandas as pd

pd.set_option('display.max_rows', None)  # Affichez toutes les lignes
# Calcul de la valeur foncière moyenne par département
df_avg_value_by_dept = df.groupby(
    'Departement')['Valeur_fonciere'].mean().reset_index()

print(df_avg_value_by_dept)
print(len(df_avg_value_by_dept), " départements différents")
# Coordonnées du centre de la France (ou un autre emplacement de votre choix)
latitude, longitude = 46.603354, 1.888334

# Créez la carte
my_map = folium.Map(location=[latitude, longitude], zoom_start=6)

# Ajoutez la carte choroplèthe en fonction de la valeur foncière moyenne par département
Choropleth(
    # Chemin vers le fichier GeoJSON des départements de la France
    geo_data='../data/departements.geojson',
    name='choropleth',
    data=df_avg_value_by_dept,
    columns=['Departement', 'Valeur_fonciere'],
    # Clé pour faire correspondre les données GeoJSON avec le DataFrame
    key_on='feature.properties.code',
    # Palette de couleurs (vous pouvez choisir d'autres palettes)
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Valeur fonciere moyenne par département'
).add_to(my_map)

# Enregistrez la carte en tant que fichier HTML
my_map.save('map_departements.html')


   Departement  Valeur_fonciere
0           01    232173.000631
1           02    148041.112690
2           03    146906.136835
3           04    213793.683335
4           05    235042.920033
5           06    323630.512454
6           07    197536.617678
7           08    146166.853935
8           09    157269.692093
9           10    164573.249340
10          11    180017.558909
11          12    166715.962161
12          13    301692.871097
13          14    216023.859978
14          15    150337.682209
15          16    158788.417051
16          17    219852.135772
17          18    144575.235190
18          19    155614.558086
19          20    279386.863205
20          21    207108.207835
21          22    179041.373193
22          23    129020.131111
23          24    176251.721839
24          25    207211.290161
25          26    224327.316767
26          27    183585.733327
27          28    185910.842679
28          29    186058.537606
29          2A    292293.100328
30      

In [44]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="my_geocoder")

# Remplacez le code postal correct de la ville voulue et vérifier les coordonnées
code_postal = "2200"
adresse = f"SOISSONS, {code_postal}, France"

# Utilisez Nominatim pour obtenir les coordonnées géographiques
location = geolocator.geocode(adresse)

# Affichez les coordonnées géographiques (latitude et longitude) de "AVERMES"
if location:
    print(f"Latitude: {location.latitude}, Longitude: {location.longitude}")
else:
    print("Coordonnées non trouvées pour l'adresse spécifiée.")


Latitude: 49.3837621, Longitude: 3.327613
